## Solve Model balancing problem with kinetic parameters as variables

In [1]:
import numpy as np
from model_balancing import ModelBalancing, Q_
import cvxpy as cp

In [2]:
use_regulation = True
args = {}

# choose solver

if cp.MOSEK in cp.installed_solvers():
    args["solver"] = cp.MOSEK
else:
    #args["solver"] = cp.SCS
    args["solver"] = cp.ECOS

In [3]:
# Network topology

args["S"] = np.matrix(
    [
        [-1, 0, 0],
        [1, -1, -1],
        [0, 1, 1]
    ], dtype=float
)

if use_regulation:
    args["A_act"] = np.matrix(
        [
            [0, 0, 1],
            [0, 0, 0],
            [0, 0, 0]
        ], dtype=float
    )
    args["A_inh"] = np.matrix(
        [
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0]
        ], dtype=float
    )
else:
    args["A_act"] = np.matrix(
        [
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0]
        ], dtype=float
    )
    args["A_inh"] = np.matrix(
        [
            [0, 0, 0],
            [0, 0, 0],
            [0, 0, 0]
        ], dtype=float
    )

In [4]:
# Priors for Kinetic and Thermodynamic parameters

args["Keq_gmean"] = Q_([1, 1, 1], "")  # geometric mean (assuming a standard concentration of 1M)
args["Keq_ln_cov"] = np.array(
    [
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
    ]
)  # log-scale covariance

args["kcatf_gmean"] = Q_([20.0, 8.0, 10.0], "1/s")  # geometric mean
args["kcatf_ln_cov"] = np.array(
    [
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
    ]
) # log-scale covariance

args["kcatr_gmean"] = Q_([3.5e-3, 6.2e4, 3.1e-1], "1/s")  # geometric mean
args["kcatr_ln_cov"] = np.array(
    [
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
    ]
) # log-scale covariance

# the K parameters are only for existing connections in the S, A_act and A_inh matrices.
# the order of values is "metabolite-first".
args["Km_gmean"] = Q_([1e-2, 1e-4, 1e-4, 1e-3, 1e-1, 1e-1], "M")
args["Km_ln_cov"] = np.array(
    [
        [1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1],
    ]
) # log-scale covariance

args["Ka_gmean"] = Q_([1e-3, 5e-3], "M")
args["Ka_ln_cov"] = np.array(
    [
        [1, 0],
        [0, 1],
    ]
)

args["Ki_gmean"] = Q_([1e-3], "M")
args["Ki_ln_cov"] = np.array(
    [
        [1],
    ]
)

In [5]:
# condition dependent data (columns represent conditions)

args["fluxes"] = Q_(
    [
        [2.0, 1.5, 2.5, 2.5],
        [1.0, 1.0, 2.0, 0.5],
        [1.0, 0.5, 0.5, 2.0],
    ], "mM/s"
)

args["conc_enz_gmean"] = Q_(
    [
        [1e-3, 1e-3, 1e-3, 3e-3],
        [2e-3, 3e-3, 1e-3, 2e-3],
        [1e-3, 2e-3, 2e-3, 1e-3],
    ], "M")  # geometric mean

args["conc_enz_gstd"] = np.array(
    [
        [1.1, 1.1, 1.1, 1.1],
        [1.1, 1.1, 1.1, 1.1],
        [1.1, 1.1, 1.1, 1.1],
    ]
)  # geometric standard deviation

args["conc_met_gmean"] = Q_(
    [
        [3e-3, 4e-3, 9e-4, 3e-4],
        [2e-3, 3e-3, 6e-4, 2e-4],
        [1e-3, 2e-3, 3e-4, 1e-4],
    ], "M"
)  # geometric mean

args["conc_met_gstd"] = np.array(
    [
        [1.1, 1.1, 1.1, 1.1],
        [1.1, 1.1, 1.1, 1.1],
        [1.1, 1.1, 1.1, 1.1],
    ]
)  # geometric standard deviation

In [6]:
args["rate_law"] = "CM"

mb = ModelBalancing(**args)

# check that the driving forces at the mode are all positive
assert mb.is_gmean_feasible()

mb.initialize_with_gmeans()
print(f"initial total squared Z-scores = {mb.objective_value:.3f}")

mb.solve()
print(f"optimized total squared Z-scores = {mb.objective_value:.3f}")

print("-"*25 + " All Z-scores " + "-"*25)
mb.print_z_scores(5)

initial total squared Z-scores = 0.000
optimized total squared Z-scores = 0.124
------------------------- All Z-scores -------------------------
enz = 0.0
met = 0.00153
Keq = 0.05093
kcatf = 0.04698
kcatr = 0.00107
Km = 0.00513
Ka = 0.01847
Ki = 0.0


In [7]:
mb.print_status()


Metabolite concentrations (M) =
 [[3.00029259e-03 4.00033276e-03 9.01942727e-04 3.00000000e-04]
 [2.00437887e-03 3.00228805e-03 5.99376425e-04 2.00000000e-04]
 [9.98502947e-04 1.99850548e-03 3.00000000e-04 9.99999999e-05]]

Enzyme concentrations (M) =
 [[0.01668403 0.01716065 0.02289366 0.02974065]
 [0.00023024 0.00031844 0.00051347 0.00016462]
 [0.00026385 0.00014533 0.00035381 0.0062495 ]]

Driving forces (RT) =
 [[0.48980934 0.37343661 0.49509495 0.49189886]
 [0.77320412 0.48334676 0.76847907 0.7695189 ]
 [0.8908045  0.60094714 0.88607945 0.88711928]]

η(thr) =
 [[0.39 0.31 0.39 0.39]
 [0.54 0.38 0.54 0.54]
 [0.59 0.45 0.59 0.59]]

η(kin) =
 [[0.01 0.01 0.01 0.01]
 [0.95 0.97 0.86 0.67]
 [0.68 0.76 0.39 0.17]]

η(reg) =
 [[1.   1.   1.   1.  ]
 [1.   1.   1.   1.  ]
 [0.77 0.82 0.51 0.26]]




